In [106]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
import requests as rq
import random as rnd
import gmaps
import os
import json
import time

# Import API key
import config

# Configure gmaps
gkey = config.gkey
gmaps.configure(api_key=gkey)

In [107]:
notebook_path = os.getcwd()
parent_dir = notebook_path[0:-4]

In [108]:
dataset_filelist = ["1_county_level_confirmed_cases.csv","2_cases_and_deaths_by_county_timeseries.csv","weekly-claims-by-county-twc.csv","texas_county_med_income.csv"]
for csv in dataset_filelist:   
    for root, dirs, files in os.walk(parent_dir):
        if csv in files:
            if csv == dataset_filelist[0]:
                covid_dataset1 = (os.path.join(root, csv))
            elif csv == dataset_filelist[1]:
               covid_dataset2 =  (os.path.join(root, csv))
            elif csv == dataset_filelist[2]:
               unemployemet_dataset =  (os.path.join(root, csv))
            else:
                texas_median_income = (os.path.join(root, csv))

In [109]:
ConfirmedByCounty_Covid = pd.read_csv(covid_dataset1)
CaseDeathsBy_County_timeseries = pd.read_csv(covid_dataset2, parse_dates=['date'])
unemployemet_dataset = pd.read_csv(unemployemet_dataset)
texas_median_income = pd.read_csv(texas_median_income)

In [110]:
texas_median_income['Median Household Income'] = texas_median_income['Median Household Income'].str.replace('$','').str.replace(',','').astype(float)
texas_median_income.describe()

,Median Household Income
count,254.000000
mean,51301.976378
std,12220.638423
min,30007.000000
25%,43253.750000
50%,48541.500000
75%,56199.250000
max,102858.000000


In [111]:
texas_low_income = texas_median_income.loc[(texas_median_income['Median Household Income'] <= 57000) & (texas_median_income['Median Household Income'] > 36000)]
texas_low_income.describe()

,Median Household Income
count,189.000000
mean,46768.005291
std,5124.925818
min,36176.000000
25%,42701.000000
50%,46912.000000
75%,50539.000000
max,56848.000000


In [112]:
texas_verylow_income = texas_median_income.loc[(texas_median_income['Median Household Income'] <= 36000) & (texas_median_income['Median Household Income'] > 26000)]
texas_verylow_income.describe()

,Median Household Income
count,9.000000
mean,32908.000000
std,2329.002254
min,30007.000000
25%,30490.000000
50%,33160.000000
75%,35156.000000
max,35881.000000


In [113]:
texas_exlow_income = texas_median_income.loc[(texas_median_income['Median Household Income'] <= 26000)]
texas_exlow_income.head()

,County,Median Household Income


In [114]:
Texas_ConfirmedByCounty_Covid = ConfirmedByCounty_Covid.loc[ConfirmedByCounty_Covid['state'] == 'Texas'].dropna(subset=['lat','lon'])
Texas_ConfirmedByCounty_Covid.head()

,last_update,location_type,state,county_name,county_name_long,fips_code,lat,lon,NCHS_urbanization,total_population,confirmed,confirmed_per_100000,deaths,deaths_per_100000
2599,2020-11-13 00:26:19 UTC,county,Texas,Anderson,"Anderson, Texas, US",48001.0,31.815347,-95.653548,Micropolitan,57863.0,3035,5245.15,42,72.59
2600,2020-11-13 00:26:19 UTC,county,Texas,Andrews,"Andrews, Texas, US",48003.0,32.304686,-102.637655,Micropolitan,17818.0,781,4383.21,11,61.74
2601,2020-11-13 00:26:19 UTC,county,Texas,Angelina,"Angelina, Texas, US",48005.0,31.254573,-94.609015,Micropolitan,87607.0,2380,2716.68,93,106.16
2602,2020-11-13 00:26:19 UTC,county,Texas,Aransas,"Aransas, Texas, US",48007.0,28.105562,-96.999505,Medium metro,24763.0,369,1490.13,18,72.69
2603,2020-11-13 00:26:19 UTC,county,Texas,Archer,"Archer, Texas, US",48009.0,33.615700,-98.687546,Small metro,8789.0,187,2127.66,1,11.38


In [115]:
CaseDeathsBy_County_timeseries = CaseDeathsBy_County_timeseries.set_index('date')

In [116]:
columns_todrop = []
Texas_CaseDeathsBy_County_timeseries = CaseDeathsBy_County_timeseries.loc[CaseDeathsBy_County_timeseries['state'] == 'Texas']
for column in Texas_CaseDeathsBy_County_timeseries.columns:
    if column.startswith('new'):
        columns_todrop.append(column)
for drop in columns_todrop:
    Texas_CaseDeathsBy_County_timeseries = Texas_CaseDeathsBy_County_timeseries.drop(columns=drop)
Texas_CaseDeathsBy_County_timeseries.head()

,uid,location_type,fips_code,location_name,state,total_population,cumulative_cases,cumulative_cases_per_100_000,cumulative_deaths,cumulative_deaths_per_100_000
date,,,,,,,,,,
2020-01-22,84048001,county,48001.0,Anderson,Texas,57863.0,0,0.0,0,0.0
2020-01-23,84048001,county,48001.0,Anderson,Texas,57863.0,0,0.0,0,0.0
2020-01-24,84048001,county,48001.0,Anderson,Texas,57863.0,0,0.0,0,0.0
2020-01-25,84048001,county,48001.0,Anderson,Texas,57863.0,0,0.0,0,0.0
2020-01-26,84048001,county,48001.0,Anderson,Texas,57863.0,0,0.0,0,0.0


In [118]:
test_copy_unemployment = unemployemet_dataset.copy()
test_copy_unemployment = test_copy_unemployment.transpose()
test_copy_unemployment

,0,1,2,3,4,5,6,7,8,9,...,244,245,246,247,248,249,250,251,252,253
County,Anderson,Andrews,Angelina,Aransas,Archer,Armstrong,Atascosa,Austin,Bailey,Bandera,...,Willacy,Williamson,Wilson,Winkler,Wise,Wood,Yoakum,Young,Zapata,Zavala
10/31/2020,42,17,77,32,3,1,46,28,3,23,...,24,422,28,7,74,31,14,16,20,11
10/24/2020,79,17,89,22,6,NaN,36,23,1,13,...,51,409,34,8,65,33,8,17,27,18
10/17/2020,54,17,80,40,5,1,35,26,4,24,...,43,434,30,15,45,26,6,17,27,12
10/10/2020,63,30,98,18,10,2,40,32,3,11,...,24,484,41,11,57,38,6,19,22,16
10/3/2020,64,25,103,26,6,1,55,37,5,18,...,15,574,41,12,79,45,9,11,19,16
9/26/2020,64,21,100,35,5,1,56,30,4,22,...,26,564,39,18,59,46,7,18,29,11
9/19/2020,61,18,119,25,4,NaN,70,29,3,25,...,23,526,42,13,59,29,5,7,23,21
9/12/2020,66,32,107,31,5,NaN,67,42,8,4,...,19,524,41,8,51,56,12,13,23,20
9/5/2020,85,22,171,50,6,3,96,45,9,41,...,29,800,54,19,86,83,10,22,34,18


In [119]:
locations = Texas_ConfirmedByCounty_Covid[['lat','lon']]
confirmed = Texas_ConfirmedByCounty_Covid['confirmed']

In [120]:
zero_coor = (31.81,-95.65)
figure_layout = {'width': '1050px', 'margin': '0 auto 0 auto'}
# Plot Heatmap
fig = gmaps.figure(center=zero_coor,zoom_level=5.5,layout=figure_layout,map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=confirmed,
                                 dissipating=False, max_intensity=.5,
                                 point_radius=.5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='1050px'))

In [ ]:
%reset